# CutOutSemantic

In [ ]:
#| default_exp cutoutsemantic

In [ ]:
#| export
# library
from semantic_segmentation_augmentations.holemakertechnique import HoleMakerTechnique
from semantic_segmentation_augmentations.regionmodifier import RegionModifier
from semantic_segmentation_augmentations.iholesfilling import HolesFilling

# others
import random
import torch
from fastai.basics import *
import numpy as np

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
class CutOutSemantic(HolesFilling):
    "Defines the amount of holes, the class to be occluded, the technique used to make them and the probability of apply the technique."
    def __init__(self,
                 holes_num = 1, # The amount of holes to make.
                 occlusion_class = -1, # The class to remove. If -1, selects it randomly in each use.
                 modifier: "RegionModifier" = None, # The modifier that defines the traditional augments to apply to the selected regions.
                 hole_maker: "HoleMakerTechnique" = None, # The strategy used to make the holes.
                 p = 0.5): # The probability of applying this technique.
        super().__init__(modifier, hole_maker)
        self.holes_num = holes_num
        self.occlusion_class = occlusion_class
        self.p = p

    def before_batch(self):
        "Applies the CutOut technique with semantic information (only applies the CutOut to a selected class)."
        for image, mask in zip(self.x, self.y):
            if random.random() < self.p:
                shape = image.shape[1:]
                for _ in range(self.holes_num):
                    xhole, yhole = self.make_hole(mask)
                    occlusion_value = self.occlusion_class if self.occlusion_class != -1 else random.randint(1, len(mask.unique()))
                    sub_image, sub_mask = TensorBase(image[:, yhole, xhole]), TensorBase(mask[yhole, xhole])
                    replacement_mask = sub_mask == occlusion_value
                    sub_image[:, replacement_mask] = torch.min(image)
                    sub_mask[replacement_mask] = 0
                    self.fill_hole(image, mask, xhole, yhole, [sub_image, sub_mask])

The default technique used to make those holes is the `HoleMakerRandom` technique.

`CutOut` technique was defined and implemented [here](https://arxiv.org/abs/1708.04552).

In [ ]:
show_doc(CutOutSemantic.before_batch)

---

### CutOutSemantic.before_batch

>      CutOutSemantic.before_batch ()

Applies the CutOut technique with semantic information (only applies the CutOut to a selected class).

In [ ]:
#| hide
import nbdev
nbdev.nbdev_export()